# s-QCT interpolation

The following is a notebook that showcases QCT based interpolation as per the work done in ***Efficient quantum interpolation of natural data***.

In [ ]:
from qjpeg_class import qjpeg_compression, qjpeg_interpolation
import numpy as np
from skimage import data
from skimage import io
from skimage.transform import rescale
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import cv2
from skimage.transform.radon_transform import fft
import matplotlib.pyplot as plt
import matplotlib as mpl
from qft_class import qft_interpolation_2d
%matplotlib inline

In [ ]:
def mpl_params():
    mpl.rcParams['axes.linewidth'] = 2.5
    mpl.rcParams['xtick.top'] = True
    mpl.rcParams['ytick.right'] = True
    mpl.rcParams['xtick.direction'] = 'in'
    mpl.rcParams['ytick.direction'] = 'in'
    mpl.rcParams['xtick.major.size'] = 5
    mpl.rcParams['ytick.major.size'] = 5
    mpl.rcParams['xtick.labelsize'] = 'x-large'
    mpl.rcParams['ytick.labelsize'] = 'x-large'
    mpl.rcParams['axes.grid'] = False
    mpl.rcParams['text.usetex'] = True
    mpl.rcParams['font.weight'] = 'bold'
    mpl.rcParams['legend.fontsize'] = 'xx-large'
    mpl.rcParams['legend.frameon'] = False

In [ ]:
def compare(image, subspace, m=1, v=False):
    image_s = cv2.resize(image, (image.shape[0]//2**m, image.shape[1]//2**m), interpolation = cv2.INTER_CUBIC)
    image_f = cv2.resize(image_s, image.shape, interpolation = cv2.INTER_CUBIC)
    qft_up = qft_interpolation_2d(image_s, upscale_factor=m)
    image_qft = qft_up()
    qjpeg_up = qjpeg_interpolation(image_s, subspace=subspace, m=m)
    image_qjpeg = qjpeg_up()
    qct_up = qjpeg_interpolation(image_s, subspace=int(np.ceil(np.log2(image_s.shape[0]))), m=m)
    image_qct = qct_up()
    p_s = psnr(image, image_f)
    s_s = ssim(image, image_f)
    p_qft = psnr(image, np.minimum(np.around(image_qft), 255).astype('uint8'))
    s_qft = ssim(image, np.minimum(np.around(image_qft), 255).astype('uint8'))
    p_qjpeg = psnr(image, np.minimum(np.around(image_qjpeg), 255).astype('uint8'))
    s_qjpeg = ssim(image, np.minimum(np.around(image_qjpeg), 255).astype('uint8'))
    p_qct = psnr(image, np.minimum(np.around(image_qct), 255).astype('uint8'))
    s_qct = ssim(image, np.minimum(np.around(image_qct), 255).astype('uint8'))
    if v:
        print('PSNR and SSIM for the upscaled images:\n')
        print(f'QJPEG properties: subspace = {subspace}\n')
        print(f'Bicubic downscale and bicubic upscale - PSNR: {p_s} SSIM: {s_s}\n')
        print(f'Bicubic downscale and qft upscale - PSNR: {p_qft} SSIM: {s_qft}\n')
        print(f'Bicubic downscale and qct upscale - PSNR: {p_qct} SSIM: {s_qct}\n')
        print(f'Bicubic downscale and qjpeg upscale - PSNR: {p_qjpeg} SSIM: {s_qjpeg}\n')
    return [[p_s, s_s], [p_qft, s_qft], [p_qct, s_qct], [p_qjpeg, s_qjpeg]]

In [ ]:
mpl_params()

**s-DCT interpolation of grayscale images**

In [ ]:
image = data.camera()

In [ ]:
upscale_factor = 2

**s = n-DCT interpolation**

In [ ]:
qct_up = qjpeg_interpolation(image, subspace=int(np.ceil(np.log2(image.shape[0]))), m=upscale_factor)

In [ ]:
%%time
interpolated_image = qct_up()

In [ ]:
image_plot = interpolated_image.copy()
image_plot[:image.shape[0], :image.shape[1]] = image

In [ ]:
fig = plt.figure(figsize=(8, 8), dpi=300)
plt.imshow(image_plot, cmap='gray')
fig.tight_layout()
#plt.savefig(f'qct-bw.png', dpi=300, format='png', bbox_inches='tight')
plt.show()

**s = 3-DCT interpolation**

In [ ]:
s = 3

In [ ]:
qjpeg_up = qjpeg_interpolation(image, subspace=s, m=upscale_factor)

In [ ]:
%%time
interpolated_image = qjpeg_up()

In [ ]:
image_plot = interpolated_image.copy()
image_plot[:image.shape[0], :image.shape[1]] = image

In [ ]:
fig = plt.figure(figsize=(8, 8), dpi=300)
plt.imshow(image_plot, cmap='gray')
fig.tight_layout()
#plt.savefig(f'cam-bw-2.png', dpi=300, format='png', bbox_inches='tight')
plt.show()

**s-DCT interpolation of RGB images**

In [ ]:
image = data.astronaut()

In [ ]:
upscale_factor = 2

**s = n-DCT interpolation**

In [ ]:
qct_up = qjpeg_interpolation(image, subspace=int(np.ceil(np.log2(image.shape[0]))), m=upscale_factor)

In [ ]:
%%time
interpolated_image = qct_up()

In [ ]:
image_plot = interpolated_image.copy()
image_plot[:image.shape[0], :image.shape[1],:] = image

In [ ]:
fig = plt.figure(figsize=(8, 8), dpi=300)
plt.imshow(image_plot, cmap='gray')
fig.tight_layout()
#plt.savefig(f'qct-rgb.png', dpi=300, format='png', bbox_inches='tight')
plt.show()

**s = 3-DCT interpolation**

In [ ]:
s = 3

In [ ]:
qjpeg_up = qjpeg_interpolation(image, subspace=s, m=upscale_factor)

In [ ]:
%%time
interpolated_image = qjpeg_up()

In [ ]:
image_plot = interpolated_image.copy()
image_plot[:image.shape[0], :image.shape[1],:] = image

In [ ]:
fig = plt.figure(figsize=(8, 8), dpi=300)
plt.imshow(image_plot, cmap='gray')
fig.tight_layout()
#plt.savefig(f'qjpeg-rgb.png', dpi=300, format='png', bbox_inches='tight')
plt.show()

**Comparison of Bicubic, QFT, n-DCT and 3-DCT interpolation**

In [ ]:
image = data.camera()

In [ ]:
compare(image, 3, m=1, v=True)